In [1]:
from korean_crawling_bundles import *
from konlpy.tag import *
import datetime
from IPython.display import display

import re
import numpy as np
import pandas as pd
from pprint import pprint
from konlpy.tag import Hannanum

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

stopwords는 불필요한 단어, 즉 조사나 관사들을 없애는 툴이다.

In [2]:
stop_words = list(pd.read_csv('kor_stop_words.csv').T.iloc[:1,:].values[0])
stop_words[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

# 전체 데이터프레임 병합하기

In [3]:
naver_df = pd.read_csv('{}_with_link_naver.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False))
daum_df = pd.read_csv('{}_with_link_daum.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False))
daum_df = daum_df.loc[:,['title','link','content']]

df = pd.concat([naver_df,daum_df],axis=0)
df.reset_index(drop=True,inplace=True)
df.tail()

,title,link,content
784,유류세 인하 일주일..휘발유 110원·경유 73원 내려,http://m.finance.daum.net/m/news/news_content....,【서울=뉴시스】김지은 기자 = 휘발유와 경유 등에 붙는 세금이 15% 인하된지 일주...
785,[단독]'라면 마케팅' 비용 메우려 과자 54개 제품 가격 올리는 농심(종합),http://m.finance.daum.net/m/news/news_content....,[아시아경제 이선애 기자] 주요 식품 가격이 줄줄이 오르고 있는 가운데 과자 가격도...
786,10월 코스닥 개인 거래비중 80%..약 20년만에 최저,http://m.finance.daum.net/m/news/news_content....,(서울=연합뉴스) 권수현 기자 = 국내 증시가 글로벌 금융위기 이후 최악의 급락을 ...
787,"[단독]새우깡도 오른다..농심, 과자 20개 브랜드 '가격 최대 9.1% 인상'",http://m.finance.daum.net/m/news/news_content....,[아시아경제 이선애 기자] 주요 식품 가격이 줄줄이 오르고 있는 가운데 과자 가격도...
788,대의원에게 소주 1박스·쌀 1포대 전달..조합 임원 '당선무효형',http://m.finance.daum.net/m/news/news_content....,(춘천=연합뉴스) 이재현 기자 = 대의원을 찾아가 소주 1박스와 쌀 1포대를 기부한...


In [4]:
append_ls,append_idx = [],[]
for idx,i in enumerate(df['title']) : 
    if i not in append_ls:
        append_ls.append(i)
        append_idx.append(idx)

In [5]:
df = df.loc[append_idx]
# df.reset_index(drop=True,inplace=True)
df.tail()

,title,link,content
729,"광주 도시철도 2호선 공론화추진위 ""건설 재개 권고""",http://m.finance.daum.net/m/news/news_content....,(광주=연합뉴스) 김재선 기자 = 광주시 도시철도 2호선 공론화추진위원회가 현재 일...
731,"기업 절반이상 ""채용공고 '정보 비공개 관행' 남아있어""",http://m.finance.daum.net/m/news/news_content....,구인구직 매칭 플랫폼 사람인이 기업 429개를 대상으로 ‘채용공고 비공개 관행’에 ...
732,"홍남기 후보자 ""고용 상황 엄중..통계 관심 있게 지켜볼 것""",http://m.finance.daum.net/m/news/news_content....,【세종=뉴시스】장서우 기자 = 지난 11일부터 청문회 준비를 시작한 홍남기 경제부총...
733,무역전쟁 속 中 '내수 파워' 보여준 '솔로의 날',http://m.finance.daum.net/m/news/news_content....,(상하이=연합뉴스) 차대운 특파원 = 중국 최대 전자 상거래 업체인 알리바바의 '소...
734,"홍남기 ""고용상황 엄중히 생각..잠재성장률 끌어 올려야""",http://m.finance.daum.net/m/news/news_content....,(서울=뉴스1) 김혜지 기자 = 홍남기 부총리 겸 기획재정부 장관 내정자는 12일 ...


In [6]:
df.fillna('0',inplace=True)

In [7]:
df['content'] = [i.replace('  ','') for i in df['content'].values]

________

In [8]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# # ▶ 이후로 나온 단어들은 기사의 author 에 관한 내용으로 지워준다.
# data = [data[i][:re.search('▶',data[i]).start()] if '▶' in i else data[i] for i in range(len(data))]

pprint(data[:1])

['KEB하나은행은 한국능률협회컨설팅(KMAC) 주관 2018년 판매서비스 만족도(KSSI) 은행(대출) 부문에서 5년 연속 1위에 '
 '선정됐다고 13일 밝혔다.지난 7월 한국산업 서비스품질지수(KSQI) 고객접점 부문에서 3년 연속 은행산업 부문 1위 선정에 따라 손님 '
 '만족도 및 서비스 품질 직원 역량에서 서비스 경쟁력을 높히 평가받은데 따른 것이다.이는 그룹의 핵심 가치인 ‘손님의 기쁨’을 최우선으로 '
 '하는 소비자 중심의 경영 문화를 위해 전 임직원이 현장과 손님 중심으로 철저히 노력한 결과라고 하나은행 측은 '
 '설명했다.한국능률협회컨설팅(KMAC)이 주관하는 판매서비스 만족도(KSSI)는 손님과 대면하는 상품 판매 인력의 질적 능력(상담태도 '
 '카운셀링 파워 업무지식 등)에 대해 영업점에서 해당 서비스를 직접 경험한 소비자가 평가한 세일즈 만족도 조사다.함영주 KEB하나은행장은 '
 '“이번 5년 연속 수상은 손님의 기쁨을 최우선 가치로 두고 노력한 결과로 한결 같은 신뢰와 사랑을 보내주신 손님들께 진심으로 '
 '감사드린다”며 “앞으로도 변화와 혁신을 통해 손님의 불편은 없애고 편리함은 높여서 금융의 새로운 가치를 제공할 수 있도록 노력하겠다”고 '
 '밝혔다.한편 KEB하나은행은 신혼부부 및 결혼예정자를 위한 ‘신혼부부 전세론’에 이어 다자녀가구를 위한 ‘다둥이 전세론’을 출시해 가정의 '
 '시작에서 자녀 양육까지 소비자 경험을 바탕으로 한 맞춤형 대출상품 라인업을 구축했다. 또 비대면 편의성을 높이기 위해 모바일 대출 신청 '
 '프로세스를 구축하는 등 소비자 니즈를 반영한 상품과 서비스를 지속적으로 선보이고 있다.또 지난 8월 출범한 하나금융그룹의 '
 '‘손님불편제거위원회’를 통해 손님 관점의 채널별 불편 사항 불합리한 관행을 사전에 제거하고 관련 제도 및 프로세스 시스템 등을 손님의 '
 '입장에서 재점검하고 있으며 디지털 금융 혁신에 발맞춘 디지털 전환과 현장 중심 손님 가치 중심의 상품 및 서비스 개발을 주도하고 있

# tokenization process

- 정규식 표현을 통해서 문장 내에 이메일과 기타 특수 문자들을 없애주었지만, 여전히 난잡해보인다.
- LDA 알고리즘을 사용하기 위해서는, 문장들을 단어들의 묶음으로 변환시켜주는 과정이 필요하다.
- 이러한 과정을 Tokenization 이라고 한다.

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
        # deacc=True removes punctuations
        # 구두점(말끝에 찍는 쉼표나 점들을 의미) 을 없애주는 것이다.

data_words = list(sent_to_words(data))

print(data_words[:1])

[['keb하나은행은', '한국능률협회컨설팅', 'kmac', '주관', '판매서비스', '만족도', 'kssi', '은행', '대출', '부문에서', '연속', '위에', '선정됐다고', '밝혔다', '지난', '한국산업', '서비스품질지수', 'ksqi', '고객접점', '부문에서', '연속', '은행산업', '부문', '선정에', '따라', '손님', '만족도', '서비스', '품질', '직원', '역량에서', '서비스', '경쟁력을', '높히', '평가받은데', '따른', '것이다', '이는', '그룹의', '핵심', '가치인', '손님의', '기쁨', '최우선으로', '하는', '소비자', '중심의', '경영', '문화를', '위해', '임직원이', '현장과', '손님', '중심으로', '철저히', '노력한', '결과라고', '하나은행', '측은', '설명했다', '한국능률협회컨설팅', 'kmac', '주관하는', '판매서비스', '만족도', 'kssi', '손님과', '대면하는', '상품', '판매', '인력의', '질적', '능력', '상담태도', '카운셀링', '파워', '업무지식', '대해', '영업점에서', '해당', '서비스를', '직접', '경험한', '소비자가', '평가한', '세일즈', '만족도', '조사다', '함영주', 'keb하나은행장은', '이번', '연속', '수상은', '손님의', '기쁨을', '최우선', '가치로', '두고', '노력한', '결과로', '한결', '같은', '신뢰와', '사랑을', '보내주신', '손님들께', '진심으로', '감사드린다', '앞으로도', '변화와', '혁신을', '통해', '손님의', '불편은', '없애고', '편리함은', '높여서', '금융의', '새로운', '가치를', '제공할', '있도록', '노력하겠다', '밝혔다', '한편', 'keb하나은행은', '신혼부부', '결혼예정자를', '위한', '신혼부부', '전세론', '이어', '다자녀가구를', '위한', '다둥이', '전세론', '

### Bigram , Trigram 모델 만들기
- Bigram : 문서에서 함께 자주 등장하는 2개의 단어
- Trigram : 문서에서 함께 자주 등장하는 3개의 단어
- ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.
- Phrases : 모델을 빌드한다.
- min_count , threshold : Pharases 의 중요한 두 개의 파라미터
    - min_count (float, optional) : Ignore all words and bigrams with total collected count lower than this value.
    - threshold (float, optional) : Represent a score threshold for forming the phrases (higher means fewer phrases). A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold. Heavily depends on concrete scoring-function, see the scoring parameter.

In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/home/hskimim/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['keb하나은행은', '한국능률협회컨설팅_kmac', '주관', '판매서비스_만족도_kssi', '은행', '대출', '부문에서_연속', '위에', '선정됐다고_밝혔다', '지난', '한국산업', '서비스품질지수', 'ksqi', '고객접점', '부문에서_연속', '은행산업', '부문', '선정에', '따라', '손님', '만족도', '서비스', '품질', '직원', '역량에서', '서비스', '경쟁력을', '높히', '평가받은데', '따른', '것이다', '이는', '그룹의', '핵심', '가치인', '손님의', '기쁨', '최우선으로', '하는', '소비자', '중심의', '경영', '문화를', '위해', '임직원이', '현장과', '손님', '중심으로', '철저히', '노력한', '결과라고', '하나은행', '측은', '설명했다', '한국능률협회컨설팅_kmac', '주관하는', '판매서비스_만족도_kssi', '손님과', '대면하는', '상품', '판매', '인력의', '질적', '능력', '상담태도', '카운셀링', '파워', '업무지식', '대해', '영업점에서', '해당', '서비스를', '직접', '경험한', '소비자가', '평가한', '세일즈', '만족도', '조사다', '함영주', 'keb하나은행장은', '이번', '연속', '수상은', '손님의', '기쁨을', '최우선', '가치로', '두고', '노력한', '결과로', '한결', '같은', '신뢰와', '사랑을', '보내주신', '손님들께', '진심으로', '감사드린다', '앞으로도', '변화와', '혁신을', '통해', '손님의', '불편은', '없애고', '편리함은', '높여서', '금융의', '새로운', '가치를', '제공할', '있도록_노력하겠다', '밝혔다', '한편', 'keb하나은행은', '신혼부부', '결혼예정자를', '위한', '신혼부부', '전세론', '이어', '다자녀가구를', '위한', '다둥이', '전세론', '출시해', '가정의', '시작에서', '자녀', '양육까

In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized = data_words_bigrams.copy()

print(data_lemmatized[:1])

[['keb하나은행은', '한국능률협회컨설팅_kmac', '주관', '판매서비스_만족도', 'kssi', '은행', '대출', '부문에서_연속', '위에', '선정됐다고_밝혔다', '지난', '한국산업', '서비스품질지수', 'ksqi', '고객접점', '부문에서_연속', '은행산업', '부문', '선정에', '손님', '만족도', '서비스', '품질', '직원', '역량에서', '서비스', '경쟁력을', '높히', '평가받은데', '따른', '것이다', '이는', '그룹의', '핵심', '가치인', '손님의', '기쁨', '최우선으로', '하는', '소비자', '중심의', '경영', '문화를', '위해', '임직원이', '현장과', '손님', '중심으로', '철저히', '노력한', '결과라고', '하나은행', '측은', '설명했다', '한국능률협회컨설팅_kmac', '주관하는', '판매서비스_만족도', 'kssi', '손님과', '대면하는', '상품', '판매', '인력의', '질적', '능력', '상담태도', '카운셀링', '파워', '업무지식', '대해', '영업점에서', '해당', '서비스를', '직접', '경험한', '소비자가', '평가한', '세일즈', '만족도', '조사다', '함영주', 'keb하나은행장은', '연속', '수상은', '손님의', '기쁨을', '최우선', '가치로', '두고', '노력한', '결과로', '한결', '같은', '신뢰와', '사랑을', '보내주신', '손님들께', '진심으로', '감사드린다', '앞으로도', '변화와', '혁신을', '통해', '손님의', '불편은', '없애고', '편리함은', '높여서', '금융의', '새로운', '가치를', '제공할', '있도록_노력하겠다', '밝혔다', '한편', 'keb하나은행은', '신혼부부', '결혼예정자를', '위한', '신혼부부', '전세론', '이어', '다자녀가구를', '위한', '다둥이', '전세론', '출시해', '가정의', '시작에서', '자녀', '양육까지', '

- LDA 모델에 들어가야 하는 두 가지 입력변수는 딕셔너리와(id2word) 코퍼스(corpus)이다.
- gensim 은 문서 내에 있는 단어별로 유니크한 아이디를 할당해준다.
- 아래의 각각의 엘리먼트 튜플당 의미하는 것은 [word_id,word_frequency] 이다.

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 3), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 3), (82, 1), (83, 4), (84, 1), (85, 1), (86, 1), (87, 4), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

만약 해당 id에 속한 단어를 보고싶으면

In [14]:
id2word[0]

'keb하나은행은'

위의 표는 컴퓨터가 읽기 쉽게끔 만들어준 것이고, Counter 객체처럼 사람이 읽기 쉽게 만든 것은 아래와 같다.

In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('keb하나은행은', 2),
  ('keb하나은행장은', 1),
  ('ksqi', 1),
  ('kssi', 2),
  ('가정의', 1),
  ('가치', 1),
  ('가치로', 1),
  ('가치를', 1),
  ('가치인', 1),
  ('감사드린다', 1),
  ('같은', 1),
  ('개발을', 1),
  ('것이다', 1),
  ('결과라고', 1),
  ('결과로', 1),
  ('결혼예정자를', 1),
  ('경영', 1),
  ('경쟁력을', 1),
  ('경험을', 1),
  ('경험한', 1),
  ('고객접점', 1),
  ('관련', 1),
  ('관점의', 1),
  ('관행을', 1),
  ('구축하는', 1),
  ('구축했다', 1),
  ('그룹의', 1),
  ('금융', 1),
  ('금융의', 1),
  ('기쁨', 1),
  ('기쁨을', 1),
  ('노력한', 2),
  ('높여서', 1),
  ('높이기_위해', 1),
  ('높히', 1),
  ('능력', 1),
  ('니즈를', 1),
  ('다둥이', 1),
  ('다자녀가구를', 1),
  ('대면하는', 1),
  ('대출', 2),
  ('대출상품', 1),
  ('대해', 1),
  ('두고', 1),
  ('등을', 1),
  ('디지털', 2),
  ('따른', 1),
  ('라인업을', 1),
  ('만족도', 2),
  ('맞춤형', 1),
  ('모바일', 1),
  ('무단_전재', 1),
  ('문화를', 1),
  ('바탕으로', 1),
  ('반영한', 1),
  ('발맞춘', 1),
  ('밝혔다', 1),
  ('변화와', 1),
  ('보내주신', 1),
  ('부문', 1),
  ('부문에서_연속', 2),
  ('불편', 1),
  ('불편은', 1),
  ('불합리한', 1),
  ('비대면', 1),
  ('사랑을', 1),
  ('사전에', 1),
  ('사항', 1),
  ('상담태도', 1),
  ('상품',

여태까지 해온 것이 LDA 모델 생성에 필요한 것들을 전부 한 것이다. 코퍼스와 딕셔너리를 생성한 것에 더해서, 우리는 몇 개의 토픽을 할당할 것인지에 대한 결정을 해주어야 한다.

- alpha , eta 는 토픽들의 떨어진 정도(sparsity)에 영향을 끼치는 하이퍼 파라미터이다. 도큐먼트에 따르면, 디폴트값은 1.0/num_topics prior 이다.
- chunksize 는 각각의 training chunk 에 사용될 문서의 갯수를 의미한다. 확실하지는 않지만, batch_size 와 유사한 의미를 갖는 것로 해석된다.

    - IN ADDITION : Text chunking, also referred to as shallow parsing, is a task that follows Part-Of-Speech Tagging and that adds more structure to the sentence. The result is a grouping of the words in “chunks”.


In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

위의 모델을 통해 반환되는 것은 토픽의 수는 20개이고 각각의 키워드(단어들의 집합)와 키워드들 간의 조합이 특정한 토픽의 가중치를 정해주는데 기여하는 것이다.

- lda_model 객체에 print_topics 메소드를 operating 하면, 각각의 키워드들이 토픽에 기여하는 가중치(importance)를 알 수 있다.
- 0부터 19까지 총 20개에 해당하는 토픽이 있는 것을 알 수 있고, 각각의 토픽에 위치해있는 키워드들과 이들 키워드들이 해당 토픽에서 가지는 중요도가 순서대로 나와있다.

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"기자" + 0.006*"있는" + 0.004*"사람들이" + 0.004*"가명ㆍ_씨는" + 0.004*"경제가" + '
  '0.004*"마스크를" + 0.004*"정부가" + 0.004*"한다" + 0.004*"이날" + 0.004*"것이"'),
 (1,
  '0.005*"않는" + 0.005*"위해" + 0.004*"복수응답" + 0.004*"기업" + 0.004*"있었다" + '
  '0.004*"밝혔다" + 0.003*"비공개" + 0.003*"구체적인" + 0.003*"공개하지" + 0.003*"지원자가"'),
 (2,
  '0.004*"분기" + 0.004*"지난해" + 0.004*"올해" + 0.004*"것으로" + 0.004*"이후" + '
  '0.003*"기자" + 0.003*"지난" + 0.003*"억원" + 0.003*"중국" + 0.003*"있는"'),
 (3,
  '0.011*"소비" + 0.007*"중국" + 0.005*"중국의" + 0.005*"도시철도_호선" + 0.004*"대한" + '
  '0.004*"건설" + 0.004*"솽스이" + 0.004*"억위안" + 0.004*"앞서" + 0.003*"통해"'),
 (4,
  '0.005*"있는" + 0.003*"통해" + 0.002*"대한" + 0.002*"지난" + 0.002*"좋은" + 0.002*"기자" '
  '+ 0.002*"현재" + 0.002*"위해" + 0.002*"되는" + 0.002*"일정"')]


In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.10303474720549

Coherence Score:  0.2944995586404243


pyLDAvis 만큼 jupyter notebook에서 LDA랄 잘 작동하면서 시각화하는 툴도 없다.

In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      31.614117        1       1 -0.068651 -0.221559
4      23.320744        1       2 -0.015992 -0.019915
1      16.271082        1       3 -0.002343  0.034202
3      14.894658        1       4 -0.155921  0.164974
0      13.899398        1       5  0.242906  0.042298, topic_info=      Category        Freq      Term       Total  loglift  logprob
term                                                              
9060   Default  174.000000        소비  174.000000  30.0000  30.0000
2575   Default  218.000000        중국  218.000000  29.0000  29.0000
369    Default  149.000000        분기  149.000000  28.0000  28.0000
12416  Default  107.000000       중국의  107.000000  27.0000  27.0000
33401  Default   78.000000   도시철도_호선   78.000000  26.0000  26.0000
33471  Default   76.000000      복수응답   76.000000  25.0000  25.0000
997    Default  125.000000        않는  125.000000  24.0000  24.0000
33107  Default   67.000000    가명ㆍ_씨는   67.000000  23.0000  23.0000
18062  Default   72.000000      사람들이   72.000000  22.0000  22.0000
23450  Default   66.000000       솽스이   66.000000  21.0000  21.0000
13943  Default   70.000000        건설   70.000000  20.0000  20.0000
5770   Default   65.000000       경제가   65.000000  19.0000  19.0000
871    Default  102.000000        기업  102.000000  18.0000  18.0000
10382  Default   60.000000       억위안   60.000000  17.0000  17.0000
384    Default  168.000000        억원  168.000000  16.0000  16.0000
33166  Default   56.000000      마스크를   56.000000  15.0000  15.0000
33177  Default  140.000000     미세먼지가  140.000000  14.0000  14.0000
12290  Default   60.000000       비공개   60.000000  13.0000  13.0000
24817  Default   58.000000      공개하지   58.000000  12.0000  12.0000
27340  Default   58.000000      지원자가   58.000000  11.0000  11.0000
33505  Default   57.000000      채용공고   57.000000  10.0000  10.0000
13666  Default   52.000000      후보자는   52.000000   9.0000   9.0000
1853   Default  108.000000       있었다  108.000000   8.0000   8.0000
23456  Default   53.000000     알리바바의   53.000000   7.0000   7.0000
4766   Default   91.000000        들어   91.000000   6.0000   6.0000
843    Default   87.000000      구체적인   87.000000   5.0000   5.0000
33176  Default   87.000000  미세먼지_농도가   87.000000   4.0000   4.0000
2463   Default   61.000000        있게   61.000000   3.0000   3.0000
13659  Default   46.000000       청문회   46.000000   2.0000   2.0000
141    Default  184.000000        통해  184.000000   1.0000   1.0000
...        ...         ...       ...         ...      ...      ...
33209   Topic5   22.468961       씁쓸해   23.033797   1.9485  -6.5118
14165   Topic5   34.585162     끌어올리는   35.529197   1.9464  -6.0806
6931    Topic5   23.353727        가고   23.953725   1.9480  -6.4732
18062   Topic5   67.619574      사람들이   72.356953   1.9056  -5.4101
28488   Topic5   23.046852        쓰지   23.645217   1.9477  -6.4864
5770    Topic5   60.712415       경제가   65.741427   1.8937  -5.5178
2812    Topic5   36.249593        씨는   39.391696   1.8902  -6.0336
3561    Topic5   34.930116        관심   38.115554   1.8861  -6.0706
2463    Topic5   50.030780        있게   61.944549   1.7597  -5.7113
7864    Topic5   34.661331        기본   39.808310   1.8349  -6.0784
13652   Topic5   25.097050       자체를   26.468636   1.9201  -6.4012
17294   Topic5   36.562328        고용   44.103871   1.7858  -6.0250
32155   Topic5   22.896277       박미주   23.656747   1.9407  -6.4930
17030   Topic5   34.974257       대책을   43.481981   1.7556  -6.0694
3801    Topic5   55.339613       정부가   90.223247   1.4845  -5.6105
5802    Topic5   34.986553        때는   45.567385   1.7091  -6.0690
6011    Topic5   27.059065        인근   30.972233   1.8383  -6.3260
1246    Topic5   53.524535        이날  105.406212   1.2956  -5.6438
826     Topic5   53.014230        것이  112.177140   1.2238  -5.6534
192     Topic5   96.811072        기자  351.192169   0.6848  -5.0512
268     Topic5   91.050989  

- Gensim 의 LDA 알고리즘보다 Mallet 버젼이 더 나은 퀄리티를 보여준다.
- https://www.machinelearningplus.com/wp-content/uploads/2018/03/mallet-2.0.8.zip 깔고 해당 경로를 아래에 넣어주면 된다.

In [20]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/home/hskimim/Documents/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5, id2word=id2word)
# Show Topics
pprint(ldamallet.show_topics(formatted=False))


[(0,
  [('통해', 0.01312482546774644),
   ('밝혔다', 0.01154239970213162),
   ('위해', 0.008749883645164293),
   ('있는', 0.008237922368053617),
   ('기자', 0.006795122405287164),
   ('다양한', 0.006795122405287164),
   ('재배포_금지', 0.006515870799590431),
   ('위한', 0.005864283719631388),
   ('금지', 0.004933445033975612),
   ('있도록', 0.004514567625430513)]),
 (1,
  [('대한', 0.007534741021476268),
   ('관계자는', 0.004556939180653312),
   ('등에', 0.004060638873849486),
   ('지난', 0.004060638873849486),
   ('등을', 0.003789929615592853),
   ('경우', 0.003699693196173976),
   ('이를', 0.003474102147626782),
   ('위해', 0.003474102147626782),
   ('말했다', 0.003474102147626782),
   ('현재', 0.003338747518498466)]),
 (2,
  [('억원', 0.012524084778420038),
   ('분기', 0.010872557115331681),
   ('지난해', 0.00945040829433893),
   ('올해', 0.007890632168088815),
   ('이후', 0.00541334067345628),
   ('지난', 0.004771079915588586),
   ('관계자는', 0.004633452610331223),
   ('기자', 0.004587576841912102),
   ('기준', 0.00449582530507386),
   ('지난달', 0.003

In [21]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.3268225261780463


### 항상 더 우월한 것은 아닌듯

#### 해당 문장에서 지배적인 토픽을 찾기
- 토픽 모델링의 주요 활용점은 해당 문서의 토픽이 무엇이냐에 관한 것이다.
- 이를 알아내기 위해서는 해당 문서에서 가장 기여를 많이 한, 즉 중요도가 가장 높은 토픽의 넘버를 찾아야 한다.
- format_topics_sentences() 메소드는 보여지는 테이블로 훌륭하게 정보를 병합해준다.

In [22]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet , corpus=corpus, texts=data)

#ldamallet 파라미터를 optimal_params 로 바꿔주는 시퀀스로 업데이트할 것!

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.6703,"통해, 밝혔다, 위해, 있는, 기자, 다양한, 재배포_금지, 위한, 금지, 있도록",KEB하나은행은 한국능률협회컨설팅(KMAC) 주관 2018년 판매서비스 만족도(KS...
1,1,0.0,0.6216,"통해, 밝혔다, 위해, 있는, 기자, 다양한, 재배포_금지, 위한, 금지, 있도록",한화생명은 제 4회 한-아세안 청년 혁신포럼에 참여한 11개국의 청년 스타트업 대표...
2,2,2.0,0.5817,"억원, 분기, 지난해, 올해, 이후, 지난, 관계자는, 기자, 기준, 지난달",[헤럴드경제=신소연 기자]신한금융지주로 주인이 바뀔 오렌지라이프(구 ING생명)의 ...
3,3,0.0,0.6154,"통해, 밝혔다, 위해, 있는, 기자, 다양한, 재배포_금지, 위한, 금지, 있도록",(서울=뉴스1 ) 박주평 기자 = BC카드가 베트남 현지 은행과 협력해 QR코드 등...
4,4,0.0,0.5840,"통해, 밝혔다, 위해, 있는, 기자, 다양한, 재배포_금지, 위한, 금지, 있도록",[아시아경제 박지환 기자] 한화생명은 제4회 한-아세안 청년 혁신포럼에 참여한 11...
5,5,0.0,0.6929,"통해, 밝혔다, 위해, 있는, 기자, 다양한, 재배포_금지, 위한, 금지, 있도록",BC카드는 베트남 리엔비엣포스트은행(LienVietPostBank)과 함께 베트남 ...
6,6,2.0,0.5182,"억원, 분기, 지난해, 올해, 이후, 지난, 관계자는, 기자, 기준, 지난달",오렌지라이프는 오는 23일부터 사흘 간 경주 블루원 디아너스 CC에서 자사가 4년 ...
7,7,0.0,0.5525,"통해, 밝혔다, 위해, 있는, 기자, 다양한, 재배포_금지, 위한, 금지, 있도록",(서울=연합뉴스) 구정모 기자 = BC카드는 리엔비엣포스트은행과 베트남 결제플랫폼의...
8,8,2.0,0.6539,"억원, 분기, 지난해, 올해, 이후, 지난, 관계자는, 기자, 기준, 지난달",태풍과 폭염 연초 한파 등 자연 재해에 따른 손해율 상승과 자동차 정비수가 인상을 ...
9,9,1.0,0.4858,"대한, 관계자는, 등에, 지난, 등을, 경우, 이를, 위해, 말했다, 현재",◆인터넷전문은행 특례법에서의 최대주주 전횡 방지책지난 칼럼에서 인터넷전문은행 특례법...


In [23]:
recommendation_df = df_dominant_topic.sort_values(by='Topic_Perc_Contrib',ascending=False)
recommendation_df.reset_index(drop=True,inplace=True)
recommendation_df

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,346,3.0,0.8469,"지난, 미국, 기자, 것으로, 대한, 시장, 달러, 따르면, 해당, 대상으로",벌집계좌 이용 거래소도 설립…신상품 출시 거래소는 당국 경고에 폐업 결정(서울=연합...
1,675,3.0,0.8463,"지난, 미국, 기자, 것으로, 대한, 시장, 달러, 따르면, 해당, 대상으로",(서울=연합뉴스) 구정모 김경윤 기자 = 당국이 올 1월 거래 실명제를 도입한 이후...
2,274,4.0,0.8138,"중국, 있는, 이라고, 말했다, 그는, 이라며, 한다, 글로벌, 정부가, 최근",한국이 미국과 중국의 무역분쟁 직격탄을 맞고 있다는 분석이 나왔다. 기존 수출산업을...
3,392,2.0,0.8081,"억원, 분기, 지난해, 올해, 이후, 지난, 관계자는, 기자, 기준, 지난달",(서울=뉴스1) 김영신 기자 = 올해 3분기 국내 11개 손해보험사 자동차보험 누적...
4,689,2.0,0.8048,"억원, 분기, 지난해, 올해, 이후, 지난, 관계자는, 기자, 기준, 지난달",(서울=뉴스1) 김영신 기자 = 올해 3분기 국내 11개 손해보험사 자동차보험 누적...
5,244,4.0,0.7936,"중국, 있는, 이라고, 말했다, 그는, 이라며, 한다, 글로벌, 정부가, 최근",[헤럴드경제]미국과 중국간 무역분쟁으로 한국이 직격탄을 맞고 있다는 금융당국 분석이...
6,673,1.0,0.7924,"대한, 관계자는, 등에, 지난, 등을, 경우, 이를, 위해, 말했다, 현재",【서울=뉴시스】김형섭 기자 = 삼성바이오로직스의 분식회계 의혹에 오는 14일 종지부...
7,232,0.0,0.7910,"통해, 밝혔다, 위해, 있는, 기자, 다양한, 재배포_금지, 위한, 금지, 있도록",[머니투데이 한은정 기자] [상담 및 자문서비스 중심의 금융센터 역할 수행] BNK...
8,94,2.0,0.7872,"억원, 분기, 지난해, 올해, 이후, 지난, 관계자는, 기자, 기준, 지난달",[머니투데이 전혜영 기자] [설계사 35% 이직 등 위해 등록 말소 말소된 설계사 ...
9,699,4.0,0.7868,"중국, 있는, 이라고, 말했다, 그는, 이라며, 한다, 글로벌, 정부가, 최근",(서울=연합뉴스) 김동현 기자 = 성윤모 산업통상자원부 장관이 12일 박용만 대한상...


In [24]:
duplicate_ls = []
duplicate_idx = []
for idx,val in enumerate(recommendation_df['Dominant_Topic'].values):
    if val not in duplicate_ls:
        duplicate_ls.append(val)
        duplicate_idx.append(idx)
        if len(duplicate_ls) == 5 : break

In [25]:
print(duplicate_ls)
print(duplicate_idx)

[3.0, 4.0, 2.0, 1.0, 0.0]
[0, 2, 3, 6, 7]


In [26]:
display(recommendation_df.loc[duplicate_idx])
display(df.loc[recommendation_df.loc[duplicate_idx]['Document_No'].values])

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,346,3.0,0.8469,"지난, 미국, 기자, 것으로, 대한, 시장, 달러, 따르면, 해당, 대상으로",벌집계좌 이용 거래소도 설립…신상품 출시 거래소는 당국 경고에 폐업 결정(서울=연합...
2,274,4.0,0.8138,"중국, 있는, 이라고, 말했다, 그는, 이라며, 한다, 글로벌, 정부가, 최근",한국이 미국과 중국의 무역분쟁 직격탄을 맞고 있다는 분석이 나왔다. 기존 수출산업을...
3,392,2.0,0.8081,"억원, 분기, 지난해, 올해, 이후, 지난, 관계자는, 기자, 기준, 지난달",(서울=뉴스1) 김영신 기자 = 올해 3분기 국내 11개 손해보험사 자동차보험 누적...
6,673,1.0,0.7924,"대한, 관계자는, 등에, 지난, 등을, 경우, 이를, 위해, 말했다, 현재",【서울=뉴시스】김형섭 기자 = 삼성바이오로직스의 분식회계 의혹에 오는 14일 종지부...
7,232,0.0,0.7910,"통해, 밝혔다, 위해, 있는, 기자, 다양한, 재배포_금지, 위한, 금지, 있도록",[머니투데이 한은정 기자] [상담 및 자문서비스 중심의 금융센터 역할 수행] BNK...


,title,link,content
346,"BNK부산銀, 디지털 금융 강화한 미래형 영업점 개점",https://m.news.naver.com/read.nhn?oid=014&aid=...,BNK부산은행은 구서동지점과 남양산지점에 전략적 핵심과제인 채널혁신과 영업방식의 변...
274,"카드사 노조, 일방적 수수료인하 반대 천막농성 돌입",https://m.news.naver.com/read.nhn?oid=015&aid=...,[ 차은지 기자] 카드사 노조가 일방적 카드수수료 인하를 반대하는 천막농성에 돌입했...
392,"신한은행, 베트남 은행 대상 '리스크 관리체계 구축' 모범사례 소개",https://m.news.naver.com/read.nhn?oid=119&aid=...,신한은행은 지난 8일 베트남에서 열린 베트남 중앙은행 신한베트남은행 및 금융감독원 ...
673,"""트럼프, 조선•반도체로 보호무역 확대할 것""",http://m.finance.daum.net/m/news/news_content....,무역협회 ‘미중 통상분쟁 전망’ 국제포럼도널드 트럼프 미국 대통령이 그간 수입 철강...
232,"신한은행, 고용노동부 퇴직연금사업자 평가서 최다부문 우수",https://m.news.naver.com/read.nhn?oid=119&aid=...,신한은행은 2018 고용노동부 퇴직연금사업자 평가에서 총 7개 부문 중 6개 부문(...


In [27]:
recommendation_df = df.loc[recommendation_df.loc[duplicate_idx]['Document_No'].values].iloc[:,:2]
recommendation_df

,title,link
346,"BNK부산銀, 디지털 금융 강화한 미래형 영업점 개점",https://m.news.naver.com/read.nhn?oid=014&aid=...
274,"카드사 노조, 일방적 수수료인하 반대 천막농성 돌입",https://m.news.naver.com/read.nhn?oid=015&aid=...
392,"신한은행, 베트남 은행 대상 '리스크 관리체계 구축' 모범사례 소개",https://m.news.naver.com/read.nhn?oid=119&aid=...
673,"""트럼프, 조선•반도체로 보호무역 확대할 것""",http://m.finance.daum.net/m/news/news_content....
232,"신한은행, 고용노동부 퇴직연금사업자 평가서 최다부문 우수",https://m.news.naver.com/read.nhn?oid=119&aid=...


In [28]:
recommendation_df['link'].values

array(['https://m.news.naver.com/read.nhn?oid=014&aid=0004124818&sid1=101&sid2=259&mode=LS2D',
       'https://m.news.naver.com/read.nhn?oid=015&aid=0004043957&sid1=101&sid2=259&mode=LS2D',
       'https://m.news.naver.com/read.nhn?oid=119&aid=0002292852&sid1=101&sid2=259&mode=LS2D',
       'http://m.finance.daum.net/m/news/news_content.daum?type=main&sub_type=&docid=MD20181112171803886&section=&limit=30&page=9',
       'https://m.news.naver.com/read.nhn?oid=119&aid=0002292909&sid1=101&sid2=259&mode=LS2D'],
      dtype=object)

# Telegram Service

In [29]:
import telegram
import pickle

my_token = pickle.load( open('pw.pickle', 'rb'))
my_token = my_token

bot = telegram.Bot(token = my_token) #봇을 생성합니다.

bot.sendMessage(chat_id='@SSU_TOP_5', text='안녕하세요. 좋은 아침입니다. {}토픽 별 기사 5개 추천해드리겠습니다.'.format(\
str(datetime.datetime.now().month)+' 월 ' + str(datetime.datetime.now().day)+' 일 '))

for i in range(len(recommendation_df)) : 
    recommendation = recommendation_df.iloc[i].values[1]
    bot.sendMessage(chat_id='@SSU_TOP_5', text=recommendation) #@bill_chat 으로 메세지를 보냅니다.

________________

#### 각각의 토픽을 대표하는 문서찾기
- 가끔 토픽 키워드(단어)는 단지 토픽들을 구성하는 것에 그치지 않는 경우가 있다.
- 토픽을 이해하는 것을 넘어서, 토픽을 형성하는데 가장 많은 기여를 한 문서를 찾아낼 수도 있다.

In [50]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.8516,"기자, 있는, 한다, 대한, 경제, 정부가, 말했다, 서울, 밝혔다, 위해",(서울=연합뉴스) 김동현 기자 = 성윤모 산업통상자원부 장관이 12일 박용만 대한상...
1,1.0,0.8629,"있는, 위해, 통해, 위한, 계획이다, 블록체인, 글로벌, 예정이다, 대한, 등을",아세안 10개국 선정 대상도시(26곳). ⓒ국토부 정부가 세종과 부산을 중심으로 추...
2,2.0,0.8473,"내년, 억원, 기자, 것으로, 미국, 경우, 국내, 대한, 것이다, 이날",(서울=연합뉴스) 박용주 기자 = 미국과 중국의 무역분쟁으로 한국이 직격탄을 맞고 ...
3,3.0,0.8535,"기자, 있는, 중국, 통해, 무단_전재, 재배포_금지, 올해, 국내, 최대, 온라인",CJ제일제당이 가정간편식(HMR) 수산캔 전문 브랜드 ‘계절어보’의 라인업을 확대한...
4,4.0,0.8320,"지난해, 분기, 지난, 이후, 기자, 것으로, 올해, 서울, 말했다, 경기","[한겨레] 강신욱 통계청장은 지난해 2분기가 경기 정점이었을 가능성이 있다며, 이르..."


#### 문서를 넘어선 토픽 분배
- 마지막으로 우리는 해당 정보에서 어떤 것들이 가장 많이 거론되었는지를 토픽의 크기(volume)과 분포(distribution)로 이해할 수 있게 된다.

In [51]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.iloc[:10]

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,0.0,"기자, 있는, 한다, 대한, 경제, 정부가, 말했다, 서울, 밝혔다, 위해",257.0,0.2420
1,4.0,"지난해, 분기, 지난, 이후, 기자, 것으로, 올해, 서울, 말했다, 경기",214.0,0.2015
2,2.0,"내년, 억원, 기자, 것으로, 미국, 경우, 국내, 대한, 것이다, 이날",208.0,0.1959
3,1.0,"있는, 위해, 통해, 위한, 계획이다, 블록체인, 글로벌, 예정이다, 대한, 등을",225.0,0.2119
4,2.0,"내년, 억원, 기자, 것으로, 미국, 경우, 국내, 대한, 것이다, 이날",158.0,0.1488
5,3.0,"기자, 있는, 중국, 통해, 무단_전재, 재배포_금지, 올해, 국내, 최대, 온라인",NaN,NaN
6,3.0,"기자, 있는, 중국, 통해, 무단_전재, 재배포_금지, 올해, 국내, 최대, 온라인",NaN,NaN
7,2.0,"내년, 억원, 기자, 것으로, 미국, 경우, 국내, 대한, 것이다, 이날",NaN,NaN
8,0.0,"기자, 있는, 한다, 대한, 경제, 정부가, 말했다, 서울, 밝혔다, 위해",NaN,NaN
9,4.0,"지난해, 분기, 지난, 이후, 기자, 것으로, 올해, 서울, 말했다, 경기",NaN,NaN
